In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_all_securities_list', 'fetch_snp500_securities_list']

In [2]:
# Parameters
sec_base_url = "https://data.sec.gov/submissions/CIK{cik_str}.json"
sample_pct = 1
snp_filter = False
upstream = {
    "fetch_snp500_securities_list": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_snp500_securities_list.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/snp500_securities_list.csv",
    },
    "fetch_all_securities_list": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_all_securities_list.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/all_securities_list.csv",
    },
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_securities.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/raw/securities.csv",
}


In [3]:
import pandas as pd
import json
import requests
import time 
from pathlib import Path
from tqdm import tqdm
from src.utils import preprocess_text

Get list of publicaly traded companies, ticker and associated cik ID. We will use cik ID to pull data from SEC's [DATA APIS](https://www.sec.gov/edgar/sec-api-documentation)

In [4]:
traded_company_df = pd.read_csv(upstream["fetch_all_securities_list"]["data"], index_col=0)
snp500_company_df = pd.read_csv(upstream["fetch_snp500_securities_list"]["data"], index_col=0)

# SEC data API needs cik number to be padded by 0's to make it a length of 10 string
traded_company_df['cik'] = traded_company_df.cik_str
traded_company_df.cik_str = traded_company_df.cik_str.astype(str).str.zfill(10)
snp500_company_df['cik_str'] = snp500_company_df.CIK.astype(str).str.zfill(10)

Download dataset for publicly traded companies incl. metadata like previous name etc. See the [DEVELOPER FAQ](https://www.sec.gov/os/webmaster-faq#developers) to get more infomration about rate limit and API call requirements.

In [5]:
# snp_filter parameter controls if we want to fetch all master publicly traded securities or just the snp500 ones
if snp_filter == True:
    traded_companies = snp500_company_df.cik_str
else:
    # sample_pct is helpful parameter to help speed up development by selecting only a fraction of securities out of 12,227 master securities       
    traded_companies = traded_company_df.cik_str.sample(frac=sample_pct)

Let's Fetch SEC submission data. This API has 10 request per second limit. so it will take about 1 hour to finish.preprocess_text

In [6]:
# SEC data API requires explicit setting of User-agent as email address, otehrwise we will get 404 error
headers = {
  'User-Agent': 'vgaurav@umich.edu'
}

df_lst = []
counter = 0
#  let's reuse connection pool by using same session.
session = requests.Session()

for cik_str in tqdm(traded_companies):
    url = sec_base_url.format(cik_str = cik_str)
    # we could have used following code, but it does not work due to nested json format of payload
    # pd.read_json(url, stosample_pctptions = {'User-Agent': 'vgaurav@umich.edu'})
    response = session.get(url, headers=headers)
    data = json.loads(response.text)
    df_lst.append(pd.json_normalize(data))
    counter += 1
    # SEC API restricts 10 API request per second, so we will use counter and time to meet this restriction
    if counter % 10 == 0:
        time.sleep(1)
        
securities_df = pd.concat(df_lst)

  0%|                                                                                                                                                                                        | 0/9213 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                | 1/9213 [00:00<31:10,  4.92it/s]

  0%|                                                                                                                                                                                | 2/9213 [00:00<24:34,  6.25it/s]

  0%|                                                                                                                                                                                | 4/9213 [00:00<21:04,  7.28it/s]

  0%|                                                                                                                                                                                | 5/9213 [00:00<21:02,  7.29it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 9209/9213 [37:39<00:00,  5.30it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 9210/9213 [37:40<00:01,  2.46it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 9211/9213 [37:40<00:00,  2.97it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9213/9213 [37:40<00:00,  3.93it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9213/9213 [37:40<00:00,  4.08it/s]

In [7]:
securities_df.cik = securities_df.cik.astype(int)
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 0
Data columns (total 47 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9213 non-null   int64 
 1   entityType                                    9213 non-null   object
 2   sic                                           9213 non-null   object
 3   sicDescription                                9213 non-null   object
 4   insiderTransactionForOwnerExists              9213 non-null   int64 
 5   insiderTransactionForIssuerExists             9213 non-null   int64 
 6   name                                          9213 non-null   object
 7   tickers                                       9213 non-null   object
 8   exchanges                                     9213 non-null   object
 9   ein                                           9203 non-null   object
 10  des

In [8]:
traded_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 12160
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cik_str  9213 non-null   object
 1   ticker   9213 non-null   object
 2   title    9213 non-null   object
 3   cik      9213 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 359.9+ KB


In [9]:
merged_df = securities_df.merge(traded_company_df, on='cik', how='inner')
# merged_df = merged_df.dropna(subset=['tickers'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9213 non-null   int64 
 1   entityType                                    9213 non-null   object
 2   sic                                           9213 non-null   object
 3   sicDescription                                9213 non-null   object
 4   insiderTransactionForOwnerExists              9213 non-null   int64 
 5   insiderTransactionForIssuerExists             9213 non-null   int64 
 6   name                                          9213 non-null   object
 7   tickers                                       9213 non-null   object
 8   exchanges                                     9213 non-null   object
 9   ein                                           9203 non-null   object
 10  

In [10]:
snp500_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 503
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 500 non-null    object
 1   Security               500 non-null    object
 2   SEC filings            500 non-null    object
 3   GICS Sector            500 non-null    object
 4   GICS Sub-Industry      500 non-null    object
 5   Headquarters Location  500 non-null    object
 6   Date first added       455 non-null    object
 7   CIK                    500 non-null    int64 
 8   Founded                500 non-null    object
 9   cik_str                500 non-null    object
dtypes: int64(1), object(9)
memory usage: 43.0+ KB


In [11]:
# left join ensures we populate fields from snp 500 df where applicable
merged_securities_df = merged_df.merge(snp500_company_df, left_on='cik', right_on='CIK', how='left')
merged_securities_df = merged_securities_df[['cik','ticker', 'exchanges', 'ein', 'name','formerNames', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'sic', 'sicDescription', 'Headquarters Location']]
merged_securities_df.columns = ['cik','ticker', 'exchanges', 'ein', 'full_name','former_names', 'short_name', 'gics_sector', 'gics_sub_industry', 'sic','sic_description', 'headquarters_location']
merged_securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cik                    9213 non-null   int64 
 1   ticker                 9213 non-null   object
 2   exchanges              9213 non-null   object
 3   ein                    9203 non-null   object
 4   full_name              9213 non-null   object
 5   former_names           9213 non-null   object
 6   short_name             499 non-null    object
 7   gics_sector            499 non-null    object
 8   gics_sub_industry      499 non-null    object
 9   sic                    9213 non-null   object
 10  sic_description        9213 non-null   object
 11  headquarters_location  499 non-null    object
dtypes: int64(1), object(11)
memory usage: 935.7+ KB


In [12]:
output_file_path = product['data']
print(len(merged_securities_df))

9213


In [13]:
merged_securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/raw/securities.csv


In [14]:
merged_securities_df.sample(10)

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
3618,1819404,NRDY,"[NYSE, NYSE]",981499860,Nerdy Inc.,"[{'name': 'TPG Pace Tech Opportunities Corp.',...",NaN,NaN,NaN,7372,Services-Prepackaged Software,NaN
6995,1831828,VERA,[Nasdaq],812744449,"Vera Therapeutics, Inc.",[],NaN,NaN,NaN,2834,Pharmaceutical Preparations,NaN
8486,824410,SASR,[Nasdaq],521532952,SANDY SPRING BANCORP INC,[],NaN,NaN,NaN,6021,National Commercial Banks,NaN
8004,1411494,AAM-PA,"[NYSE, NYSE]",208880053,"Apollo Asset Management, Inc.","[{'name': 'Apollo Global Management, Inc.', 'f...",NaN,NaN,NaN,6282,Investment Advice,NaN
9199,1023198,SGNI,[OTC],541812385,"StemGen, Inc.","[{'name': 'AMASYS CORP', 'from': '1996-10-15T0...",NaN,NaN,NaN,7372,Services-Prepackaged Software,NaN
7238,1083220,XELB,[Nasdaq],760307819,"XCel Brands, Inc.","[{'name': 'NETFABRIC HOLDINGS, INC', 'from': '...",NaN,NaN,NaN,6794,Patent Owners & Lessors,NaN
709,1823239,MRVI,[Nasdaq],000000000,"MARAVAI LIFESCIENCES HOLDINGS, INC.",[],NaN,NaN,NaN,2834,Pharmaceutical Preparations,NaN
5761,949961,IOR,[NYSE],752615944,INCOME OPPORTUNITY REALTY INVESTORS INC /TX/,[],NaN,NaN,NaN,6798,Real Estate Investment Trusts,NaN
8547,717538,AROW,[Nasdaq],222448962,ARROW FINANCIAL CORP,[],NaN,NaN,NaN,6021,National Commercial Banks,NaN
3580,1557376,OCEL,[OTC],474180540,"Organicell Regenerative Medicine, Inc.",[{'name': 'Biotech Products Services & Researc...,NaN,NaN,NaN,3790,Miscellaneous Transportation Equipment,NaN


In [15]:
del securities_df, traded_company_df, traded_companies, snp500_company_df, merged_securities_df